In [5]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from allensdk.brain_observatory.ecephys import ecephys_session
# %matplotlib inline


#Note the following three lines are necessary to pull fs_utils
import sys
sys.path.append('../swdb_2019_tools')
# import fs_utils

# fix slow autocomplete
%config Completer.use_jedi = False

import platform
platstring = platform.platform()

In [6]:
if 'Darwin' in platstring:
    # OS X 
    data_root = "/Volumes/Brain2019/"
elif 'Windows'  in platstring:
    # Windows (replace with the drive letter of USB drive)
    data_root = "E:/"
elif ('amzn1' in platstring):
    # then on AWS
    data_root = "/data/"
else:
    # then your own linux platform
    # EDIT location where you mounted hard drive
    data_root = "/media/$USERNAME/Brain2019/"

manifest_path = os.path.join(data_root, "dynamic-brain-workshop/visual_coding_neuropixels/2019/manifest.json")

cache = EcephysProjectCache.fixed(manifest=manifest_path)
sessions = cache.get_sessions()

In [7]:
sessions = cache.get_sessions()
session = sessions.index[0]
session_data = cache.get_session_data(session)

In [8]:
spiketimes = session_data.spike_times

In [9]:
spiketrain = spiketimes[914613384]
train2 = spiketimes[914613386]
train3 = spiketimes[914613390]

spiketimes_list = [spiketrain,train2,train3]

In [20]:
spiketimes_list

[array([6.62989190e-01, 3.89575906e+00, 4.99826015e+00, ...,
        9.27489710e+03, 9.27502220e+03, 9.27511023e+03]),
 array([6.28122489e-01, 6.54955849e-01, 7.28389255e-01, ...,
        9.27778883e+03, 9.27783327e+03, 9.27785537e+03]),
 array([5.62389091e-01, 6.27822489e-01, 6.68522529e-01, ...,
        9.27784967e+03, 9.27787497e+03, 9.27789327e+03])]

In [23]:
def spiketimes_to_firingrate(spiketimes,startime,stoptime,binsize,binarize=False): 
    '''
    Parameters
    ==========
    spiketimes : np.array
        array of spiketimes from one cell
    starttime, stoptime : float
        bounds of time to calculate firing rate between (in seconds)
    binsize : int
        size of bins (in milliseconds); use binsize = 1000 to get firing rate in Hz
    binarize : Boolean
        Binarize bins (returns array of 1's and 0's).  Default is False
    
    Returns
    =======
    counts : np.array
        array of counts within bins
    
    Example
    =======
    # Convert spiketimes of one cell to Hz (1000 ms bins) for a sample of 100 minutes
    spiketimes_to_firingrate(spiketrain,startime=0,stoptime=60*100,binsize=1000)
    --> array([1, 0, 0, ..., 3, 4, 1])
    '''
    startime_ms = startime * 1000
    stoptime_ms = stoptime * 1000
    binrange = np.arange(start = startime_ms,stop = stoptime_ms+1, step = binsize)

    spiketimes_cell = np.asarray(spiketimes) * 1000. #spiketimes in seconds * 1000 msec/sec
    counts, bins = np.histogram(spiketimes_cell,bins = binrange)
    if binarize == True:
        #binarize the counts
        counts[counts>0] = 1
    return(counts)

In [30]:
def spiketimes_to_2D_rates(spiketimes_list,startime,stoptime,binsize,binarize=False): 
    '''
    Parameters
    ==========
    spiketimes_list : list
        List of arrays of spiketimes, where each list element is the spiketimes array of a single cell
    starttime, stoptime : float
        bounds of time to calculate firing rate between (in seconds)
    binsize : int
        size of bins (in milliseconds); use binsize = 1000 to get firing rate in Hz
    binarize : Boolean
        Binarize bins (returns array of 1's and 0's).  Default is False
    
    Returns
    =======
    spikerates_2D : np.array
        N x T array of binned counts where rows are cells and columns are binned time
    
    Example
    =======
    # Convert spiketimes of multiple cell to Hz (1000 ms bins) for a sample of 100 minutes
    spiketimes_list
    --> [array([6.62989190e-01, 3.89575906e+00, 4.99826015e+00, ...,
        9.27489710e+03, 9.27502220e+03, 9.27511023e+03]),
     array([6.28122489e-01, 6.54955849e-01, 7.28389255e-01, ...,
        9.27778883e+03, 9.27783327e+03, 9.27785537e+03]),
     array([5.62389091e-01, 6.27822489e-01, 6.68522529e-01, ...,
        9.27784967e+03, 9.27787497e+03, 9.27789327e+03])]
    
    spiketimes_to_2D_rates(spiketimes_list,startime=0,stoptime=60*100,binsize=1000)
    --> array([[ 1,  0,  0, ...,  3,  4,  1],
           [17, 18, 33, ..., 25, 26, 38],
           [ 6, 18, 15, ..., 28, 29, 28]])
    '''
    startime_ms = startime * 1000
    stoptime_ms = stoptime * 1000
    binrange = np.arange(start = startime_ms,stop = stoptime_ms+1, step = binsize)
    n_cells = len(spiketimes_list)

    spikewords_array = np.zeros([n_cells,binrange.shape[0]-1])
    for i in range(n_cells):
        spiketimes_cell = np.asarray(spiketimes_list)[i] * 1000. #spiketimes in seconds * 1000 msec/sec
        counts, bins = np.histogram(spiketimes_cell,bins = binrange)
        if binarize == True:
            #binarize the counts
            counts[counts>0] = 1
        spikewords_array[i,:] = counts
    spikerates_2D = spikewords_array.astype(np.int)
    return(spikerates_2D)

In [27]:
spiketimes_list

[array([6.62989190e-01, 3.89575906e+00, 4.99826015e+00, ...,
        9.27489710e+03, 9.27502220e+03, 9.27511023e+03]),
 array([6.28122489e-01, 6.54955849e-01, 7.28389255e-01, ...,
        9.27778883e+03, 9.27783327e+03, 9.27785537e+03]),
 array([5.62389091e-01, 6.27822489e-01, 6.68522529e-01, ...,
        9.27784967e+03, 9.27787497e+03, 9.27789327e+03])]

In [28]:
spiketimes_to_2D_rates(spiketimes_list,startime=0,stoptime=60*100,binsize=1000)

array([[ 1,  0,  0, ...,  3,  4,  1],
       [17, 18, 33, ..., 25, 26, 38],
       [ 6, 18, 15, ..., 28, 29, 28]])

In [22]:
help(spiketimes_to_firingrate)

Help on function spiketimes_to_firingrate in module __main__:

spiketimes_to_firingrate(spiketimes, startime, stoptime, binsize, binarize=False)
    Parameters
    spiketimes : np.array
        array of spiketimes from one cell
    starttime, stoptime : float
        bounds of time to calculate firing rate between (in seconds)
    binsize : int
        size of bins (in milliseconds); use binsize = 1000 to get firing rate in Hz
    binarize : Boolean
        Binarize bins (returns array of 1's and 0's).  Default is False
    
    Returns
    counts : np.array
        array of counts within bins
    
    Example
    spiketimes_to_firingrate(spiketrain,startime=0,stoptime=60*100,binsize=1000)
    array([1, 0, 0, ..., 3, 4, 1])

